<a href="https://colab.research.google.com/github/v-jhonattan/Transfer_Learning/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import tensorflow_datasets as tfds
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# 1. Carregar dataset
(ds_train_raw, ds_val_raw, ds_test_raw), ds_info = tfds.load(
    'oxford_flowers102',
    split=['train', 'validation', 'test'],
    with_info=True,
    as_supervised=True
)

In [ ]:
# 2. Extração de labels - Método 100% funcional
def extract_labels(dataset):
    labels = []
    for _, label in tfds.as_numpy(dataset):  # Itera sobre o dataset convertido para numpy
        labels.append(int(label))  # Converte explicitamente para inteiro
    return np.array(labels)

train_labels = extract_labels(ds_train_raw)
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weight_dict = dict(enumerate(class_weights))

# Verificação
print("Número de labels:", len(train_labels))
print("Primeiros 10 labels:", train_labels[:10])
print("Contagem por classe:", {i: np.sum(train_labels==i) for i in np.unique(train_labels)})

Número de labels: 1020
Primeiros 10 labels: [72 84 70 51 48 83 42 58 40 35]
Contagem por classe: {np.int64(0): np.int64(10), np.int64(1): np.int64(10), np.int64(2): np.int64(10), np.int64(3): np.int64(10), np.int64(4): np.int64(10), np.int64(5): np.int64(10), np.int64(6): np.int64(10), np.int64(7): np.int64(10), np.int64(8): np.int64(10), np.int64(9): np.int64(10), np.int64(10): np.int64(10), np.int64(11): np.int64(10), np.int64(12): np.int64(10), np.int64(13): np.int64(10), np.int64(14): np.int64(10), np.int64(15): np.int64(10), np.int64(16): np.int64(10), np.int64(17): np.int64(10), np.int64(18): np.int64(10), np.int64(19): np.int64(10), np.int64(20): np.int64(10), np.int64(21): np.int64(10), np.int64(22): np.int64(10), np.int64(23): np.int64(10), np.int64(24): np.int64(10), np.int64(25): np.int64(10), np.int64(26): np.int64(10), np.int64(27): np.int64(10), np.int64(28): np.int64(10), np.int64(29): np.int64(10), np.int64(30): np.int64(10), np.int64(31): np.int64(10), np.int64(32): np

In [ ]:
# 3. Pré-processamento para EfficientNet
def preprocess(image, label, img_size=224):
    image = tf.image.resize(image, [img_size, img_size])
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image, label


In [ ]:
# 4. Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
])


In [ ]:
# 5. Pipeline de dados
def prepare_dataset(ds, batch_size=32, augment=False):
    ds = ds.map(lambda x, y: preprocess(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                  num_parallel_calls=tf.data.AUTOTUNE)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

ds_train = prepare_dataset(ds_train_raw, augment=True)
ds_val = prepare_dataset(ds_val_raw)
ds_test = prepare_dataset(ds_test_raw)


In [ ]:
# 6. Modelo EfficientNetB0 CORRIGIDO
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

In [ ]:
# 7. Adicionando Dropout manualmente (substituto para drop_connect)
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)  # Dropout adicional
x = layers.Dense(1024, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)  # Dropout final
outputs = layers.Dense(102, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=outputs)


In [ ]:
# 8. Compilação
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



In [ ]:

# 9. Treinamento
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=20,
    class_weight=class_weight_dict,
    callbacks=[
        callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(patience=3)
    ]
)

# 10. Avaliação
test_loss, test_acc = model.evaluate(ds_test)
print(f'\nAcurácia no teste: {test_acc:.2%}\n')

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 390s 12s/step - accuracy: 0.0820 - loss: 5.2450 - val_accuracy: 0.1225 - val_loss: 4.1116 - learning_rate: 0.0010
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 425s 12s/step - accuracy: 0.5025 - loss: 2.0602 - val_accuracy: 0.2971 - val_loss: 3.0777 - learning_rate: 0.0010
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 366s 11s/step - accuracy: 0.7036 - loss: 1.1172 - val_accuracy: 0.5647 - val_loss: 1.8188 - learning_rate: 0.0010
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 386s 11s/step - accuracy: 0.8368 - loss: 0.5984 - val_accuracy: 0.6971 - val_loss: 1.2854 - learning_rate: 0.0010
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 379s 11s/step - accuracy: 0.9134 - loss: 0.3234 - val_accuracy: 0.7696 - val_loss: 0.9254 - learning_rate: 0.0010
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 382s 11s/step - accuracy: 0.9369 - loss: 0.2265 - val_accuracy: 0.7951 - val_loss: 0.8212 - learning_rate: 0.0010
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 379s 11s/step - accuracy: 0.9534 - loss: 0.1564 - 